In [10]:
import os
os.environ['COHERE_API_KEY'] = "43w9iLmkVlcDMzLiYXfPfnRgdRtG7NZDT6wCSzcK"

In [11]:
from langchain_cohere import ChatCohere
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [12]:
# creating tool
@tool
def getPower(a: int, b: int) -> int:
    """
    This function takes two integers and returns the power of a raised to b.
    """
    return a ** b;

In [13]:
print(getPower.invoke({'a': 2, 'b': 3}))

8


In [14]:
llm = ChatCohere(model='command-r')

In [15]:
# tool Binding
binded_tool = llm.bind_tools([getPower])

In [ ]:
binded_tool.invoke("how are you doing ?") # no tool is called because there is no need to call it, because output doesn't require it now. 

AIMessage(content="Hey there! I'm doing well, thank you for asking 😊 How about you?", additional_kwargs={'id': '20831709-4741-4bf5-ab84-1e13badee512', 'finish_reason': 'COMPLETE', 'content': "Hey there! I'm doing well, thank you for asking 😊 How about you?", 'token_count': {'input_tokens': 909.0, 'output_tokens': 102.0}}, response_metadata={'id': '20831709-4741-4bf5-ab84-1e13badee512', 'finish_reason': 'COMPLETE', 'content': "Hey there! I'm doing well, thank you for asking 😊 How about you?", 'token_count': {'input_tokens': 909.0, 'output_tokens': 102.0}}, id='run-ae579bb3-46f0-4f33-bdd2-0ae225a3511a-0', usage_metadata={'input_tokens': 909, 'output_tokens': 102, 'total_tokens': 1011})

In [21]:
print(binded_tool.invoke("calculate 2^3"))
binded_tool.invoke("calculate 2^3").tool_calls # here we can see the tool got called when we asked to calculate 2^3

content='I will use the function getPower to calculate the answer to this mathematical equation.' additional_kwargs={'id': '2e917864-3e6b-4856-bb73-97c80ae3d7ae', 'finish_reason': 'TOOL_CALL', 'tool_plan': 'I will use the function getPower to calculate the answer to this mathematical equation.', 'tool_calls': [{'id': 'getPower_09p432ed66wn', 'type': 'function', 'function': {'name': 'getPower', 'arguments': '{"a":2,"b":3}'}}], 'token_count': {'input_tokens': 909.0, 'output_tokens': 63.0}} response_metadata={'id': '2e917864-3e6b-4856-bb73-97c80ae3d7ae', 'finish_reason': 'TOOL_CALL', 'tool_plan': 'I will use the function getPower to calculate the answer to this mathematical equation.', 'tool_calls': [{'id': 'getPower_09p432ed66wn', 'type': 'function', 'function': {'name': 'getPower', 'arguments': '{"a":2,"b":3}'}}], 'token_count': {'input_tokens': 909.0, 'output_tokens': 63.0}} id='run-9bbbf13d-2d1e-4a01-8908-c478925764db-0' tool_calls=[{'name': 'getPower', 'args': {'a': 2, 'b': 3}, 'id':

[{'name': 'getPower',
  'args': {'a': 2, 'b': 3},
  'id': 'getPower_w890gay40ahs',
  'type': 'tool_call'}]

In [ ]:
# tool execution
response = binded_tool.invoke("calculate 2^3")
print(response.tool_calls[0])

{'name': 'getPower', 'args': {'a': 2, 'b': 3}, 'id': 'getPower_per4j8wwk8nc', 'type': 'tool_call'}


In [24]:
response.tool_calls[0]['args']

{'a': 2, 'b': 3}

In [ ]:
getPower.invoke(response.tool_calls[0]['args']) # we can also call the tool directly with the args we got from the response, and we have do the same.

8

In [ ]:
# tool execution
getPower.invoke(response.tool_calls[0]) # we can see that when we give whole json object of tool call, it provides structured output with some additional information. (Tool Message)

ToolMessage(content='8', name='getPower', tool_call_id='getPower_per4j8wwk8nc')

In [38]:
# provide tool's reponse to llm for response generation
query = HumanMessage("calculate 15^3")

messages = [query]

response = binded_tool.invoke(messages)

messages.append(response)

In [39]:
messages

[HumanMessage(content='calculate 15^3', additional_kwargs={}, response_metadata={}),
 AIMessage(content='I will use the provided function to calculate the answer to this mathematical equation.', additional_kwargs={'id': '95712f08-be55-4e48-af5a-3639d74f65ff', 'finish_reason': 'TOOL_CALL', 'tool_plan': 'I will use the provided function to calculate the answer to this mathematical equation.', 'tool_calls': [{'id': 'getPower_q0e5r7e4sa6c', 'type': 'function', 'function': {'name': 'getPower', 'arguments': '{"a":15,"b":3}'}}], 'token_count': {'input_tokens': 910.0, 'output_tokens': 63.0}}, response_metadata={'id': '95712f08-be55-4e48-af5a-3639d74f65ff', 'finish_reason': 'TOOL_CALL', 'tool_plan': 'I will use the provided function to calculate the answer to this mathematical equation.', 'tool_calls': [{'id': 'getPower_q0e5r7e4sa6c', 'type': 'function', 'function': {'name': 'getPower', 'arguments': '{"a":15,"b":3}'}}], 'token_count': {'input_tokens': 910.0, 'output_tokens': 63.0}}, id='run-be5

In [40]:
tool_result = getPower.invoke(response.tool_calls[0])
tool_result

ToolMessage(content='3375', name='getPower', tool_call_id='getPower_q0e5r7e4sa6c')

In [41]:
messages.append(tool_result)
messages

[HumanMessage(content='calculate 15^3', additional_kwargs={}, response_metadata={}),
 AIMessage(content='I will use the provided function to calculate the answer to this mathematical equation.', additional_kwargs={'id': '95712f08-be55-4e48-af5a-3639d74f65ff', 'finish_reason': 'TOOL_CALL', 'tool_plan': 'I will use the provided function to calculate the answer to this mathematical equation.', 'tool_calls': [{'id': 'getPower_q0e5r7e4sa6c', 'type': 'function', 'function': {'name': 'getPower', 'arguments': '{"a":15,"b":3}'}}], 'token_count': {'input_tokens': 910.0, 'output_tokens': 63.0}}, response_metadata={'id': '95712f08-be55-4e48-af5a-3639d74f65ff', 'finish_reason': 'TOOL_CALL', 'tool_plan': 'I will use the provided function to calculate the answer to this mathematical equation.', 'tool_calls': [{'id': 'getPower_q0e5r7e4sa6c', 'type': 'function', 'function': {'name': 'getPower', 'arguments': '{"a":15,"b":3}'}}], 'token_count': {'input_tokens': 910.0, 'output_tokens': 63.0}}, id='run-be5

*we are trying to save a conversation history here to help llm to generate proper response.*

In [42]:
final_response = binded_tool.invoke(messages)
final_response

AIMessage(content='The answer to this equation is **3375**.', additional_kwargs={'id': 'bd616c7b-c7f4-4214-a83d-50cccc6539b1', 'finish_reason': 'COMPLETE', 'content': 'The answer to this equation is **3375**.', 'citations': [Citation(start=32, end=37, text='*3375', sources=[ToolSource(type='tool', id='getPower_q0e5r7e4sa6c:0', tool_output={'output': '3375'})], type='TEXT_CONTENT')], 'token_count': {'input_tokens': 1006.0, 'output_tokens': 56.0}}, response_metadata={'id': 'bd616c7b-c7f4-4214-a83d-50cccc6539b1', 'finish_reason': 'COMPLETE', 'content': 'The answer to this equation is **3375**.', 'citations': [Citation(start=32, end=37, text='*3375', sources=[ToolSource(type='tool', id='getPower_q0e5r7e4sa6c:0', tool_output={'output': '3375'})], type='TEXT_CONTENT')], 'token_count': {'input_tokens': 1006.0, 'output_tokens': 56.0}}, id='run-2b519255-d915-431f-8c3a-2e74e16ca15b-0', usage_metadata={'input_tokens': 1006, 'output_tokens': 56, 'total_tokens': 1062})